### 데이터 값 목록 전체

* 데이터 값 전체 자료를 csv 파일로 저장

In [2]:
import os
import pandas as pd

# 파일 저장 위치
path_dir = "D:/PythonProject/data-gatherer/dart_fs_notes/items_from_section/"
file_list = os.listdir(path_dir)

df_all = pd.DataFrame()    
for file_name in file_list:
    if file_name != 'items_all.csv' and not file_name.startswith('__'):
        df = pd.read_csv(path_dir + file_name, dtype=object)
        df_all = pd.concat([df_all, df])
    
df_all.drop_duplicates(subset=['corp_code','rpt_num'], keep='first', inplace=True, ignore_index=True)            
df_all.to_csv(path_dir + 'items_all.csv', index=False)
len(df_all['corp_code'].unique())

342

### 데이터값 결과

In [12]:
import pandas as pd
# 데이터 Loading
path_dir = "D:/PythonProject/data-gatherer/dart_fs_notes/items_from_section/"
df_all = pd.read_csv(path_dir + 'items_all.csv', dtype=object)
num_of_rpt = int(df_all.shape[0])
num_of_corp = len(df_all['corp_code'].unique())
print(f"기업 {num_of_corp}, 사업보고서 {num_of_rpt}")

기업 342, 사업보고서 2444


In [13]:
# 기업개황정보 파일 읽기
file_path = "D:/PythonProject/data-gatherer/dart_fs_notes/company/corp_info_all.csv"
df_corp_info_all = pd.read_csv(file_path, dtype=object)

In [14]:
# 데이터값과 기업개황정보 연결
df_all = pd.merge(df_all, df_corp_info_all[['corp_code', 'jurir_no', 'bizr_no', 'induty_code']], 
                  how='left', left_on='corp_code', right_on='corp_code')

In [15]:
# 제조업
df = df_all[df_all['induty_code'].str[0:2].between('10', '34', inclusive='both')]
df.shape[0]

1949

In [16]:
df['doc_result'].value_counts()

doc_result
OK                              1903
No Financial Statement Notes      46
Name: count, dtype: int64

In [17]:
df['unit'].value_counts()

unit
천원     1445
백만원     218
원       171
Name: count, dtype: int64

In [18]:
import sys
module_path = "D:\PythonProject\data-gatherer\dart_fs_notes"
sys.path.append(module_path)
import myutil

In [19]:
pd.options.display.max_colwidth

50

* 재무제표 주석 항목이 없는 경우

In [20]:
df[df.doc_title.isna()].shape[0]

46

* 문서 찾기(doc_result), 금액 찾기(item_result) 결과 통계

In [21]:
pd.options.display.max_colwidth = 100
result_chk = df[['doc_result', 'item_result']].value_counts()
df_result_chk = pd.DataFrame(result_chk)
df_result_chk.reset_index(inplace=True)
df_result_chk.to_csv("D:/PythonProject/data-gatherer/dart_fs_notes/output/" + "result_chk_" + myutil.now_dt_str() + ".csv", index=False)

### 수기 확인 필요한 목록

* Section Title은 찾았지만 금액 찾기(Value Table)는 실패한 경우

In [26]:
df_no_contents = df[df.item_result == "No Contents"]
df_no_contents = df_no_contents.sort_values(by=['corp_code'], ascending=False)
df_no_contents.to_csv("D:/PythonProject/data-gatherer/dart_fs_notes/output/" + "result_types_" + myutil.now_dt_str() + ".csv", index=False)
num_of_rpt = int(df_no_contents.shape[0])
num_of_corp = len(df_no_contents['corp_code'].unique())
print(f"Section Title은 찾았지만 금액 찾기는 실패한 경우: 기업 {num_of_corp}, 사업보고서 {num_of_rpt}")
df_no_contents.head(3)

Section Title은 찾았지만 금액 찾기는 실패한 경우: 기업 12, 사업보고서 63


,corp_code,stock_code,corp_cls,corp_name,rpt_num,rpt_name,doc_title,doc_result,section_title,unit,val_a,val_b,item_result,doc_url,jurir_no,bizr_no,induty_code
2197,00854997,137310,Y,에스디바이오센서,20210514001634,[기재정정]사업보고서 (2020.12),5. 재무제표 주석,OK,19. 확정기여형 퇴직급여제도,NaN,0.0,0.0,No Contents,http://dart.fss.or.kr/report/viewer.do?rcpNo=20210514001634&dcmNo=8070680&eleId=19&offset=824780...,1358110191121,1358613315,27112
2196,00854997,137310,Y,에스디바이오센서,20220321001483,사업보고서 (2021.12),5. 재무제표 주석,OK,20. 확정기여형 퇴직급여제도,NaN,0.0,0.0,No Contents,http://dart.fss.or.kr/report/viewer.do?rcpNo=20220321001483&dcmNo=8487583&eleId=22&offset=891696...,1358110191121,1358613315,27112
2195,00854997,137310,Y,에스디바이오센서,20230515000371,[기재정정]사업보고서 (2022.12),5. 재무제표 주석,OK,20. 확정기여형 퇴직급여제도,NaN,0.0,0.0,No Contents,http://dart.fss.or.kr/report/viewer.do?rcpNo=20230515000371&dcmNo=9272423&eleId=24&offset=873206...,1358110191121,1358613315,27112


* 금액 찾기 오류 발생 건

In [23]:
df_result_chk[df_result_chk.item_result == "Exception"]

,doc_result,item_result,count
69,OK,Exception,1


In [24]:
df_exception = df[df.item_result == "Exception"]
df_exception.to_csv("D:/PythonProject/data-gatherer/dart_fs_notes/output/" + "result_chk_exception_" + myutil.now_dt_str() + ".csv", index=False)
num_of_rpt = int(df_exception.shape[0])
num_of_corp = len(df_exception['corp_code'].unique())
print(f"금액 찾기 오류 발생 건: 기업 {num_of_corp}, 사업보고서 {num_of_rpt}")
df_exception

금액 찾기 오류 발생 건: 기업 1, 사업보고서 1


,corp_code,stock_code,corp_cls,corp_name,rpt_num,rpt_name,doc_title,doc_result,section_title,unit,val_a,val_b,item_result,doc_url,jurir_no,bizr_no,induty_code
681,00125530,005610,Y,SPC삼립,20230316001219,사업보고서 (2022.12),5. 재무제표 주석,OK,3. 순확정급여부채의 재측정요소,NaN,0.0,0.0,Exception,http://dart.fss.or.kr/report/viewer.do?rcpNo=20230316001219&dcmNo=9066329&eleId=22&offset=816226...,1101110090516,1338122244,10712


* 금액 찾기 오류 발생 건

In [25]:
str_expr = "val_a == '-1'or val_b == '-1'"
df.query(str_expr)

,corp_code,stock_code,corp_cls,corp_name,rpt_num,rpt_name,doc_title,doc_result,section_title,unit,val_a,val_b,item_result,doc_url,jurir_no,bizr_no,induty_code
